vi. Attacker relationship (how do they claim to know the victim, 1)
met online? 2) friend of a friend?, 3) met the victim in person
before)

In [12]:
import json
import os

PATH = '../data/separated by email/'

for email in os.listdir(PATH):
    print(email)
    j = json.load(open(os.path.join(PATH, email)))
    print(j['X-TIKA:content'].strip())
    break

3721.json
From Dr Kenneth

Independent Committee of

Eminent Persons (ICEP),London.

Email:kcole444@yahoo.co.uk







ATTENTION:







I am a member of Independent Committee of Eminent Persons (ICEP),London.

ICEP is charged with the responsibility of finding bank accounts in Switzerland belonging to non-Swiss indigenes, which have remained dormant since World War II. 



It may interest you to know that in July of 1997, the Swiss Banker's Association published a list of dormant accounts originally opened by non-Swiss citizens. 

These accounts had been dormant since the end of World War II (May 9, 1945). Most belonged to Holocaust victims.

The continuing efforts of the Independent Committee of Eminent Persons (ICEP)have since resulted in the discovery of additional dormant accounts -54 ,000 in December, 1999.



The published lists contain all types of dormant

accounts,including interest-bearing savings accounts,

securities accounts, safe deposit boxes, custody accounts, and non-

ix. Attacker estimated age. You will use the USC Data Science AgePredictor, here: https://github.com/USCDataScience/AgePredictor